In [1]:
import os
import pandas as pd
from dotenv import load_dotenv
from langchain.embeddings import CacheBackedEmbeddings, HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.text_splitter import CharacterTextSplitter
from langchain.docstore.document import Document
from langchain.chains import ConversationalRetrievalChain
from langchain.llms import OpenAI
from langchain.memory import ConversationBufferMemory

# Load environment variables from the .env file
load_dotenv()

# Initialize the local file store for caching
store = LocalFileStore("./cache/")

# Define the HuggingFace embedding model to use
embed_model_id = 'sentence-transformers/all-MiniLM-L6-v2'

# Initialize the core embedding model from HuggingFace
core_embeddings_model = HuggingFaceEmbeddings(
    model_name=embed_model_id
)

# Create a cache-backed embedding model
embedder = CacheBackedEmbeddings.from_bytes_store(
    core_embeddings_model, store, namespace=embed_model_id
)

# Step 1: Load the CSV file
csv_path = 'abstracts_output.csv'  # Replace with your actual CSV file path
df = pd.read_csv(csv_path)

# Step 2: Prepare documents and metadata
texts = df['text'].tolist()
metadata = [{"authors": row['authors'], "source": row['source'], "year": row['year']} for _, row in df.iterrows()]

# Convert each text and its metadata into a Document object
documents = [Document(page_content=text, metadata=meta) for text, meta in zip(texts, metadata)]

# Step 3: Split the text into chunks for processing
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
split_data = text_splitter.split_documents(documents)

# Step 4: Create the FAISS vector store from the documents and cache-backed embedder
vector_store = FAISS.from_documents(split_data, embedder)

# Set up the conversation chain with memory and retrieval
llm = OpenAI(temperature=0.1)  # Ensure the OpenAI API key is set in your environment
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)
conversation_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    chain_type="stuff",
    retriever=vector_store.as_retriever(),
    memory=memory
)

# Step 5: Generate the report and format the metadata as APA citations
def generate_report():
    try:
        # Use the conversation chain to generate a report
        result = conversation_chain({"question": "Summarize the key findings from the abstracts, use the metadata to generate APA in text citation."})
        report = result["answer"]

        # Manually attach metadata to the answer
        used_metadata = [doc.metadata for doc in split_data if doc.page_content in report]

        # Prepare the APA citation part
        citation_prompt = "Present the following author and year information as APA citations: "
        for meta in used_metadata:
            citation_prompt += f"{meta['authors']} ({meta['year']}), "

        # Generate the APA citations using the LLM
        citation_result = llm(citation_prompt.strip().rstrip(','))

        # Append the APA citations to the report
        final_report = report + "\n\n### References\n" + citation_result

        return final_report

    except Exception as e:
        return f"Error generating report: {e}"

# Call the function to generate the report
final_report = generate_report()
print(final_report)


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\hp\anaconda3\envs\vivli_2024\Lib\site-packages\IPython\core\interactiveshell.py", line 2168, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\hp\anaconda3\envs\vivli_2024\Lib\site-packages\IPython\core\ultratb.py", line 1454, in structured_traceback
    return FormattedTB.structured_traceback(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\hp\anaconda3\envs\vivli_2024\Lib\site-packages\IPython\core\ultratb.py", line 1345, in structured_traceback
    return VerboseTB.structured_traceback(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\hp\anaconda3\envs\vivli_2024\Lib\site-packages\IPython\core\ultratb.py", line 1192, in structured_traceback
    formatted_exception = self.format_exception_as_a_whole(etype, evalue, etb, number_of_lines_of_context,
                          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

In [4]:
import chardet
import pandas as pd

# Detect the encoding of the file
with open('WHO_AWARE.csv', 'rb') as f:
    result = chardet.detect(f.read())

# Use the detected encoding to read the file
df_variables = pd.read_csv('WHO_AWARE.csv', encoding=result['encoding'])


In [5]:
result['encoding']

'ISO-8859-1'

In [2]:
import requests

# Define the base URL for the openFDA API
base_url = "https://api.fda.gov/drug/event.json"

# Define the parameters for the query
params = {
    "search": "patient.drug.medicinalproduct:omeprazole",
    "count": "patient.reaction.reactionmeddrapt.exact",
    "limit": 10  # Number of results to return
}

# Make the GET request to the openFDA API
response = requests.get(base_url, params=params)

# Check if the request was successful
if response.status_code == 200:
    # Parse the JSON response
    data = response.json()

    # Print the side effects
    print("Common side effects of Ciproflocaxin:")
    for item in data['results']:
        print(f"- {item['term']} (Count: {item['count']})")
else:
    print(f"Failed to retrieve data. Status code: {response.status_code}")



Common side effects of Ciproflocaxin:
- NAUSEA (Count: 18985)
- DIARRHOEA (Count: 18747)
- FATIGUE (Count: 18561)
- DYSPNOEA (Count: 15627)
- DRUG INEFFECTIVE (Count: 15406)
- CHRONIC KIDNEY DISEASE (Count: 14776)
- HEADACHE (Count: 13744)
- PAIN (Count: 13211)
- VOMITING (Count: 12757)
- DIZZINESS (Count: 12492)


In [5]:
# Base URL for the openFDA API
base_url = "https://api.fda.gov/drug/label.json"

# Parameters for the query
params = {
    "search": "openfda.brand_name:azithromycin",
    "limit": 1  # Limit the number of results
}

# Make the GET request to the openFDA API
response = requests.get(base_url, params=params)

# Check if the request was successful
if response.status_code == 200:
    # Parse the JSON response
    data = response.json()

    # Print the drug label information
    for label in data['results']:
        print("Drug Label Information:")
        print(f"- Brand Name: {label.get('openfda', {}).get('brand_name', ['N/A'])[0]}")
        print(f"- Generic Name: {label.get('openfda', {}).get('generic_name', ['N/A'])[0]}")
        print(f"- Indications and Usage: {label.get('indications_and_usage', ['N/A'])[0]}")
        print(f"- Dosage and Administration: {label.get('dosage_and_administration', ['N/A'])[0]}")
        print(f"- Warnings: {label.get('warnings', ['N/A'])[0]}")
        print()
else:
    print(f"Failed to retrieve data. Status code: {response.status_code}")


Drug Label Information:
- Brand Name: Azithromycin
- Generic Name: AZITHROMYCIN
- Indications and Usage: 1 INDICATIONS AND USAGE Azithromycin is a macrolide antibacterial drug indicated for the treatment of patients with mild to moderate infections caused by susceptible strains of the designated microorganisms in the specific conditions listed below. Recommended dosages and durations of therapy in adult and pediatric patient populations vary in these indications. [see Dosage and Administration (2) ] Azithromycin is a macrolide antibacterial drug indicated for mild to moderate infections caused by designated, susceptible bacteria: • Acute bacterial exacerbations of chronic bronchitis in adults ( 1.1 ) • Acute bacterial sinusitis in adults ( 1.1 ) • Uncomplicated skin and skin structure infections in adults ( 1.1 ) • Urethritis and cervicitis in adults ( 1.1 ) • Genital ulcer disease in men ( 1.1 ) • Acute otitis media in pediatric patients (6 months of age and older) ( 1.2 ) • Community

In [7]:
import pandas as pd

In [3]:
import pandas as pd

# Load the dataset
df = pd.read_csv('2024_05_28 atlas_antibiotics.csv')

# Select the specified columns and those ending with _I
selected_columns = ['Species', 'Family', 'Country', 'Gender', 'Age Group', 'Speciality', 'Source', 'In / Out Patient', 'Year']
selected_columns += [col for col in df.columns if col.endswith('_I')]

# Create a new dataframe with the selected columns
df_selected = df[selected_columns]

# Rename columns to remove the _I suffix
df_selected.columns = [col[:-2] if col.endswith('_I') else col for col in df_selected.columns]


# Display the first few rows of the selected dataframe
print(df_selected.head())


                      Species                Family Country  Gender  \
0       Staphylococcus aureus    Staphylococcus spp   India    Male   
1      Pseudomonas aeruginosa  Non-Enterobacterales   India  Female   
2       Klebsiella pneumoniae      Enterobacterales   India    Male   
3       Klebsiella pneumoniae      Enterobacterales   India    Male   
4  Acinetobacter nosocomialis  Non-Enterobacterales   India  Female   

        Age Group       Speciality                 Source  In / Out Patient  \
0  65 to 84 Years  Surgery General                  Wound               NaN   
1  65 to 84 Years   Emergency Room                  Blood               NaN   
2  19 to 64 Years      Surgery ICU  Endotracheal aspirate               NaN   
3  19 to 64 Years      Surgery ICU  Endotracheal aspirate               NaN   
4  19 to 64 Years    Pediatric ICU  Endotracheal aspirate               NaN   

   Year     Amikacin  ...  Teicoplanin Tetracycline Trimethoprim sulfa  \
0  2022          NaN  ..

C:\Users\hp\AppData\Local\Temp\ipykernel_19016\889379590.py:4: DtypeWarning: Columns (15,19,20,39,40) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('2024_05_28 atlas_antibiotics.csv')


In [5]:
df_selected.to_csv('wide_atlas.csv')

In [16]:
filtered_df = df_antibiotics[(df_antibiotics['Antibiotic'] == 'Ampicillin') & (df_antibiotics['Resistance'] == 'Resistant')]


In [17]:
filtered_df

,Species,Country,Gender,Age.Group,Speciality,Source,Year,Phenotype,Antibiotic,Resistance,Infection_Diagnosis
26,Klebsiella pneumoniae,India,Male,19 to 64 Years,Surgery ICU,Endotracheal aspirate,2022,ESBL,Ampicillin,Resistant,Respiratory Infection
49,Klebsiella pneumoniae,India,Male,19 to 64 Years,Surgery ICU,Endotracheal aspirate,2022,ESBL,Ampicillin,Resistant,Respiratory Infection
84,Klebsiella pneumoniae,India,Male,19 to 64 Years,Medicine General,Urine,2022,NaN,Ampicillin,Resistant,Urinary Tract Infection
143,Escherichia coli,India,Male,19 to 64 Years,Medicine General,Wound,2022,NaN,Ampicillin,Resistant,Wound Infection
188,Escherichia coli,India,Male,19 to 64 Years,Medicine General,Urine,2022,ESBL,Ampicillin,Resistant,Urinary Tract Infection
...,...,...,...,...,...,...,...,...,...,...,...
873335,Escherichia coli,Kenya,Female,19 to 64 Years,Other,Urine,2022,NaN,Ampicillin,Resistant,Urinary Tract Infection
873367,Escherichia coli,Kenya,Female,19 to 64 Years,Surgery General,Abscess,2022,NaN,Ampicillin,Resistant,Abscess Infection
873387,Escherichia coli,Kenya,Male,19 to 64 Years,Surgery General,Abscess,2022,NaN,Ampicillin,Resistant,Abscess Infection
873407,Escherichia coli,Ghana,Female,85 and Over,Surgery General,Urine,2022,NaN,Ampicillin,Resistant,Urinary Tract Infection


In [10]:
import streamlit as st
import pandas as pd

# Function to read the CSV file
def load_data(file_path):
    # Load the dataset
    df = pd.read_csv(file_path, low_memory=False)
    # Replace spaces with dots in the column names
    df.columns = df.columns.str.replace(' ', '.')
    return df

# Path to your CSV file
file_path = 'wide_atlas.csv'

# Load the dataset
df = load_data(file_path)

In [11]:
df

,Unnamed:.0,Species,Family,Country,Gender,Age.Group,Speciality,Source,In./.Out.Patient,Year,...,Teicoplanin,Tetracycline,Trimethoprim.sulfa,Ceftolozane.tazobactam,Cefoperazone.sulbactam,Meropenem.vaborbactam,Cefpodoxime,Ceftibuten,Ceftibuten.avibactam,Tebipenem
0,0,Staphylococcus aureus,Staphylococcus spp,India,Male,65 to 84 Years,Surgery General,Wound,NaN,2022,...,Susceptible,NaN,Susceptible,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Pseudomonas aeruginosa,Non-Enterobacterales,India,Female,65 to 84 Years,Emergency Room,Blood,NaN,2022,...,NaN,NaN,NaN,Susceptible,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Klebsiella pneumoniae,Enterobacterales,India,Male,19 to 64 Years,Surgery ICU,Endotracheal aspirate,NaN,2022,...,NaN,NaN,Resistant,Susceptible,NaN,Susceptible,Resistant,Susceptible,NaN,NaN
3,3,Klebsiella pneumoniae,Enterobacterales,India,Male,19 to 64 Years,Surgery ICU,Endotracheal aspirate,NaN,2022,...,NaN,NaN,Resistant,Susceptible,NaN,Susceptible,Resistant,Susceptible,NaN,NaN
4,4,Acinetobacter nosocomialis,Non-Enterobacterales,India,Female,19 to 64 Years,Pediatric ICU,Endotracheal aspirate,NaN,2022,...,NaN,NaN,Susceptible,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55693,55693,Acinetobacter baumannii,Non-Enterobacteriaceae,Kenya,Female,19 to 64 Years,Surgery General,Skin: Other,NaN,2022,...,NaN,NaN,Resistant,NaN,NaN,NaN,NaN,NaN,NaN,NaN
55694,55694,Escherichia coli,Enterobacteriaceae,Kenya,Female,19 to 64 Years,Surgery General,Abscess,NaN,2022,...,NaN,NaN,Resistant,Intermediate,NaN,Susceptible,NaN,NaN,NaN,NaN
55695,55695,Escherichia coli,Enterobacteriaceae,Kenya,Male,19 to 64 Years,Surgery General,Abscess,NaN,2022,...,NaN,NaN,Resistant,Susceptible,NaN,Susceptible,NaN,NaN,NaN,NaN
55696,55696,Escherichia coli,Enterobacteriaceae,Ghana,Female,85 and Over,Surgery General,Urine,NaN,2022,...,NaN,NaN,Resistant,Susceptible,NaN,Susceptible,NaN,NaN,NaN,NaN


In [12]:
# Filter the dataset based on columns with 'Resistant' results
antibiotics = [
    'Amikacin', 'Amoxycillin.clavulanate', 'Ampicillin', 'Azithromycin', 'Cefepime', 
    'Cefoxitin', 'Ceftazidime', 'Ceftriaxone', 'Clarithromycin', 'Clindamycin', 
    'Erythromycin', 'Imipenem', 'Levofloxacin', 'Linezolid', 'Meropenem', 
    'Metronidazole', 'Minocycline', 'Penicillin', 'Piperacillin.tazobactam', 
    'Tigecycline', 'Vancomycin', 'Ampicillin.sulbactam', 'Aztreonam', 
    'Aztreonam.avibactam', 'Cefixime', 'Ceftaroline', 'Ceftaroline.avibactam', 
    'Ceftazidime.avibactam', 'Ciprofloxacin', 'Colistin', 'Daptomycin', 
    'Doripenem', 'Ertapenem', 'Gatifloxacin', 'Gentamicin', 'Moxifloxacin', 
    'Oxacillin', 'Quinupristin.dalfopristin', 'Sulbactam', 'Teicoplanin', 
    'Tetracycline', 'Trimethoprim.sulfa', 'Ceftolozane.tazobactam', 
    'Cefoperazone.sulbactam', 'Meropenem.vaborbactam', 'Cefpodoxime', 
    'Ceftibuten', 'Ceftibuten.avibactam', 'Tebipenem'
]

# Filter the dataset for Resistant values
df_filtered = df[df[antibiotics].eq('Resistant').any(axis=1)]

In [13]:
df_filtered

,Unnamed:.0,Species,Family,Country,Gender,Age.Group,Speciality,Source,In./.Out.Patient,Year,...,Teicoplanin,Tetracycline,Trimethoprim.sulfa,Ceftolozane.tazobactam,Cefoperazone.sulbactam,Meropenem.vaborbactam,Cefpodoxime,Ceftibuten,Ceftibuten.avibactam,Tebipenem
0,0,Staphylococcus aureus,Staphylococcus spp,India,Male,65 to 84 Years,Surgery General,Wound,NaN,2022,...,Susceptible,NaN,Susceptible,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Pseudomonas aeruginosa,Non-Enterobacterales,India,Female,65 to 84 Years,Emergency Room,Blood,NaN,2022,...,NaN,NaN,NaN,Susceptible,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Klebsiella pneumoniae,Enterobacterales,India,Male,19 to 64 Years,Surgery ICU,Endotracheal aspirate,NaN,2022,...,NaN,NaN,Resistant,Susceptible,NaN,Susceptible,Resistant,Susceptible,NaN,NaN
3,3,Klebsiella pneumoniae,Enterobacterales,India,Male,19 to 64 Years,Surgery ICU,Endotracheal aspirate,NaN,2022,...,NaN,NaN,Resistant,Susceptible,NaN,Susceptible,Resistant,Susceptible,NaN,NaN
4,4,Acinetobacter nosocomialis,Non-Enterobacterales,India,Female,19 to 64 Years,Pediatric ICU,Endotracheal aspirate,NaN,2022,...,NaN,NaN,Susceptible,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55693,55693,Acinetobacter baumannii,Non-Enterobacteriaceae,Kenya,Female,19 to 64 Years,Surgery General,Skin: Other,NaN,2022,...,NaN,NaN,Resistant,NaN,NaN,NaN,NaN,NaN,NaN,NaN
55694,55694,Escherichia coli,Enterobacteriaceae,Kenya,Female,19 to 64 Years,Surgery General,Abscess,NaN,2022,...,NaN,NaN,Resistant,Intermediate,NaN,Susceptible,NaN,NaN,NaN,NaN
55695,55695,Escherichia coli,Enterobacteriaceae,Kenya,Male,19 to 64 Years,Surgery General,Abscess,NaN,2022,...,NaN,NaN,Resistant,Susceptible,NaN,Susceptible,NaN,NaN,NaN,NaN
55696,55696,Escherichia coli,Enterobacteriaceae,Ghana,Female,85 and Over,Surgery General,Urine,NaN,2022,...,NaN,NaN,Resistant,Susceptible,NaN,Susceptible,NaN,NaN,NaN,NaN


In [19]:
antibiotic_prev = 'Amikacin'

In [20]:

# Ensure that antibiotic_prev corresponds to a column name in the dataset

# Filter the dataset where the selected antibiotic column has 'Resistant' values
filtered_df1 = df_filtered[df_filtered[antibiotic_prev] == 'Resistant']

# Display the filtered dataframe (for debugging or further processing)
print(filtered_df1)

       Unnamed:.0                     Species                  Family Country  \
21             21       Klebsiella pneumoniae        Enterobacterales   India   
22             22       Klebsiella pneumoniae        Enterobacterales   India   
51             51       Klebsiella pneumoniae        Enterobacterales   India   
54             54       Klebsiella pneumoniae        Enterobacterales   India   
55             55       Klebsiella pneumoniae        Enterobacterales   India   
...           ...                         ...                     ...     ...   
55633       55633  Acinetobacter haemolyticus  Non-Enterobacteriaceae  Uganda   
55685       55685      Pseudomonas aeruginosa  Non-Enterobacteriaceae   Kenya   
55687       55687      Pseudomonas aeruginosa  Non-Enterobacteriaceae   Kenya   
55691       55691            Escherichia coli      Enterobacteriaceae   Kenya   
55694       55694            Escherichia coli      Enterobacteriaceae   Kenya   

       Gender       Age.Gro

In [21]:
print(filtered_df1['Amikacin'])

21       Resistant
22       Resistant
51       Resistant
54       Resistant
55       Resistant
           ...    
55633    Resistant
55685    Resistant
55687    Resistant
55691    Resistant
55694    Resistant
Name: Amikacin, Length: 3985, dtype: object


In [24]:
# Convert the dataframe to long format
df_long = pd.melt(filtered_df1, 
                  id_vars=[col for col in df_filtered.columns if col not in antibiotics], 
                  value_vars=antibiotics, 
                  var_name='Antibiotics', 
                  value_name='Resistance')

In [25]:
df_long

,Unnamed:.0,Species,Family,Country,Gender,Age.Group,Speciality,Source,In./.Out.Patient,Year,Antibiotics,Resistance
0,21,Klebsiella pneumoniae,Enterobacterales,India,Male,0 to 2 Years,Pediatric ICU,Endotracheal aspirate,NaN,2022,Amikacin,Resistant
1,22,Klebsiella pneumoniae,Enterobacterales,India,Male,19 to 64 Years,Surgery ICU,Blood,NaN,2022,Amikacin,Resistant
2,51,Klebsiella pneumoniae,Enterobacterales,India,Male,19 to 64 Years,Surgery ICU,Endotracheal aspirate,NaN,2022,Amikacin,Resistant
3,54,Klebsiella pneumoniae,Enterobacterales,India,Male,19 to 64 Years,Emergency Room,Blood,NaN,2022,Amikacin,Resistant
4,55,Klebsiella pneumoniae,Enterobacterales,India,Male,85 and Over,Surgery ICU,Urine,NaN,2022,Amikacin,Resistant
...,...,...,...,...,...,...,...,...,...,...,...,...
195260,55633,Acinetobacter haemolyticus,Non-Enterobacteriaceae,Uganda,Female,19 to 64 Years,Other,Wound,NaN,2022,Tebipenem,NaN
195261,55685,Pseudomonas aeruginosa,Non-Enterobacteriaceae,Kenya,Female,0 to 2 Years,Other,Blood,NaN,2022,Tebipenem,NaN
195262,55687,Pseudomonas aeruginosa,Non-Enterobacteriaceae,Kenya,Female,0 to 2 Years,Pediatric General,Urine,NaN,2022,Tebipenem,NaN
195263,55691,Escherichia coli,Enterobacteriaceae,Kenya,Male,19 to 64 Years,Other,Gastric Abscess,NaN,2022,Tebipenem,NaN


In [26]:
import pandas as pd

# Load the dataset
def load_data(file_path):
    df = pd.read_csv(file_path, low_memory=False)
    df.columns = df.columns.str.replace(' ', '.')
    return df

# Path to your CSV file
file_path = 'wide_atlas.csv'
df = load_data(file_path)

# List of antibiotics columns
antibiotics = [
    'Amikacin', 'Amoxycillin.clavulanate', 'Ampicillin', 'Azithromycin', 'Cefepime', 
    'Cefoxitin', 'Ceftazidime', 'Ceftriaxone', 'Clarithromycin', 'Clindamycin', 
    'Erythromycin', 'Imipenem', 'Levofloxacin', 'Linezolid', 'Meropenem', 
    'Metronidazole', 'Minocycline', 'Penicillin', 'Piperacillin.tazobactam', 
    'Tigecycline', 'Vancomycin', 'Ampicillin.sulbactam', 'Aztreonam', 
    'Aztreonam.avibactam', 'Cefixime', 'Ceftaroline', 'Ceftaroline.avibactam', 
    'Ceftazidime.avibactam', 'Ciprofloxacin', 'Colistin', 'Daptomycin', 
    'Doripenem', 'Ertapenem', 'Gatifloxacin', 'Gentamicin', 'Moxifloxacin', 
    'Oxacillin', 'Quinupristin.dalfopristin', 'Sulbactam', 'Teicoplanin', 
    'Tetracycline', 'Trimethoprim.sulfa', 'Ceftolozane.tazobactam', 
    'Cefoperazone.sulbactam', 'Meropenem.vaborbactam', 'Cefpodoxime', 
    'Ceftibuten', 'Ceftibuten.avibactam', 'Tebipenem'
]

# Ensure that all relevant columns are strings
df[antibiotics] = df[antibiotics].astype(str)
df_long = pd.melt(df, id_vars=[col for col in df.columns if col not in antibiotics], 
                  value_vars=antibiotics, var_name='Antibiotics', value_name='Resistance')

# Initialize an empty list to store results
results = []

# Iterate over each pair of antibiotics
for i, antibiotic_1 in enumerate(antibiotics):
    # Filter for rows where the first antibiotic is 100% resistant
    resistant_df = df_long[df_long['Antibiotics'] == antibiotic_1]
    resistant_df = resistant_df[resistant_df['Resistance'] == 'Resistant']
    
    # If no resistance data available for this antibiotic, skip it
    if resistant_df.empty:
        continue
    
    # Check resistance in other antibiotics when the first antibiotic is 100% resistant
    for antibiotic_2 in antibiotics[i+1:]:
        resistance_in_2 = df_long[(df_long['Antibiotics'] == antibiotic_2) & 
                                  (df_long.index.isin(resistant_df.index))]
        resistance_summary = resistance_in_2['Resistance'].value_counts(normalize=True) * 100
        results.append({
            'Antibiotic 1': antibiotic_1,
            'Antibiotic 2': antibiotic_2,
            'Resistance in Antibiotic 2 when Antibiotic 1 is 100% Resistant': 
                resistance_summary.get('Resistant', 0)
        })

# Convert results into a DataFrame for better readability
results_df = pd.DataFrame(results)



In [27]:
# Display the results
results_df


,Antibiotic 1,Antibiotic 2,Resistance in Antibiotic 2 when Antibiotic 1 is 100% Resistant
0,Amikacin,Amoxycillin.clavulanate,0
1,Amikacin,Ampicillin,0
2,Amikacin,Azithromycin,0
3,Amikacin,Cefepime,0
4,Amikacin,Cefoxitin,0
...,...,...,...
878,Cefpodoxime,Ceftibuten,0
879,Cefpodoxime,Ceftibuten.avibactam,0
880,Cefpodoxime,Tebipenem,0
881,Ceftibuten,Ceftibuten.avibactam,0


In [46]:
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter

# Load the dataset
def load_data(file_path):
    df = pd.read_csv(file_path, low_memory=False)
    df.columns = df.columns.str.replace(' ', '.')
    return df

# Path to your CSV file
file_path = 'wide_atlas.csv'  # Replace with your file path
df = load_data(file_path)

# List of antibiotics columns
antibiotics = [
    'Amikacin', 'Amoxycillin.clavulanate', 'Ampicillin', 'Azithromycin', 'Cefepime', 
    'Cefoxitin', 'Ceftazidime', 'Ceftriaxone', 'Clarithromycin', 'Clindamycin', 
    'Erythromycin', 'Imipenem', 'Levofloxacin', 'Linezolid', 'Meropenem', 
    'Metronidazole', 'Minocycline', 'Penicillin', 'Piperacillin.tazobactam', 
    'Tigecycline', 'Vancomycin', 'Ampicillin.sulbactam', 'Aztreonam', 
    'Aztreonam.avibactam', 'Cefixime', 'Ceftaroline', 'Ceftaroline.avibactam', 
    'Ceftazidime.avibactam', 'Ciprofloxacin', 'Colistin', 'Daptomycin', 
    'Doripenem', 'Ertapenem', 'Gatifloxacin', 'Gentamicin', 'Moxifloxacin', 
    'Oxacillin', 'Quinupristin.dalfopristin', 'Sulbactam', 'Teicoplanin', 
    'Tetracycline', 'Trimethoprim.sulfa', 'Ceftolozane.tazobactam', 
    'Cefoperazone.sulbactam', 'Meropenem.vaborbactam', 'Cefpodoxime', 
    'Ceftibuten', 'Ceftibuten.avibactam', 'Tebipenem'
]

# For each row, find all antibiotics columns without NaN
df['Non-NaN Antibiotics'] = df.apply(lambda row: tuple(sorted([antibiotic for antibiotic in antibiotics if pd.notna(row[antibiotic])])), axis=1)

# Count the occurrence of each combination of non-NaN antibiotics
combination_counts = Counter(df['Non-NaN Antibiotics'])

# Convert to DataFrame for plotting
combination_df = pd.DataFrame(combination_counts.items(), columns=['Antibiotic Combination', 'Count'])

# Sort the DataFrame by count for better visualization
combination_df = combination_df.sort_values(by='Count', ascending=False)

# Plotting the combinations
plt.figure(figsize=(16, 12))  # Increased figure size
plt.barh(combination_df['Antibiotic Combination'].astype(str), combination_df['Count'], color='skyblue')
plt.xlabel('Count')
plt.ylabel('Antibiotic Combination')
plt.title('Distribution of Antibiotic Combinations with Data')
plt.gca().invert_yaxis()  # Invert y-axis to show the most frequent combination at the top

# Saving the plot to a file instead of showing it directly
plt.savefig('antibiotic_combinations_distribution.png')




In [47]:
combination_df

,Antibiotic Combination,Count
2,"(Amikacin, Amoxycillin.clavulanate, Ampicillin...",21630
0,"(Ceftaroline, Clindamycin, Daptomycin, Erythro...",9408
1,"(Amikacin, Aztreonam, Cefepime, Ceftazidime, C...",8749
3,"(Amikacin, Ampicillin.sulbactam, Cefepime, Cef...",5195
6,"(Ampicillin, Daptomycin, Erythromycin, Levoflo...",3598
4,"(Ceftaroline, Ceftriaxone, Clindamycin, Erythr...",3030
5,"(Clindamycin, Daptomycin, Erythromycin, Gentam...",2093
23,"(Clindamycin, Meropenem, Metronidazole, Penici...",681
11,"(Amoxycillin.clavulanate, Ampicillin, Ampicill...",530
12,(),408


In [52]:
import pandas as pd

# Load the dataset
file_path = 'wide_atlas.csv'
df = pd.read_csv(file_path)

# Extract antibiotic columns (assuming resistance is marked as 'Resistant')
antibiotic_columns = df.columns[100:]  # Adjust based on actual columns

# Initialize an empty DataFrame to store coresistance rates
coresistance_matrix = pd.DataFrame(index=antibiotic_columns, columns=antibiotic_columns)

# Calculate coresistance rates
for ab1 in antibiotic_columns:
    for ab2 in antibiotic_columns:
        # Filter rows where both antibiotics are resistant
        resistant_both = df[(df[ab1] == 'Resistant') & (df[ab2] == 'Resistant')]
        total_isolates = df.shape[0]
        coresistance_rate = len(resistant_both) / total_isolates
        coresistance_matrix.loc[ab1, ab2] = coresistance_rate

# Convert the matrix to a numeric type
coresistance_matrix = coresistance_matrix.astype(float)

# Display the coresistance matrix
import ace_tools as tools; tools.display_dataframe_to_user(name="Coresistance Matrix", dataframe=coresistance_matrix)


C:\Users\hp\AppData\Local\Temp\ipykernel_19016\2150533925.py:5: DtypeWarning: Columns (15,25) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


ModuleNotFoundError: No module named 'ace_tools'

In [53]:
coresistance_matrix

""


In [56]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load the dataset
file_path = 'wide_atlas.csv'
df = pd.read_csv(file_path)

# Extract antibiotic columns (assuming resistance is marked as 'Resistant')
antibiotic_columns = df.columns[10:]  # Adjust based on actual columns

# Create the subfolder if it doesn't exist
output_dir = 'heat_maps'
os.makedirs(output_dir, exist_ok=True)

# Iterate over each antibiotic and create a heatmap
for antibiotic in antibiotic_columns:
    # Initialize an empty DataFrame to store coresistance rates
    coresistance_rates = pd.Series(index=antibiotic_columns, dtype=float)
    
    for ab in antibiotic_columns:
        # Filter rows where both antibiotics are resistant
        resistant_both = df[(df[antibiotic] == 'Resistant') & (df[ab] == 'Resistant')]
        total_isolates = df.shape[0]
        coresistance_rate = len(resistant_both) / total_isolates
        coresistance_rates[ab] = coresistance_rate
    
    # Create a heatmap for the current antibiotic
    plt.figure(figsize=(10, 8))
    sns.heatmap(coresistance_rates.values.reshape(1, -1), annot=True, cmap='coolwarm', 
                xticklabels=antibiotic_columns, yticklabels=[antibiotic])
    plt.title(f'Coresistance Heatmap for {antibiotic}')
    plt.xlabel('Antibiotics')
    plt.ylabel(f'{antibiotic}')
    
    # Save the heatmap as an image file in the specified subfolder
    plt.savefig(os.path.join(output_dir, f'coresistance_heatmap_{antibiotic}.png'))
    plt.close()  # Close the figure to free up memory


C:\Users\hp\AppData\Local\Temp\ipykernel_19016\38647658.py:8: DtypeWarning: Columns (15,25) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


In [57]:
coresistance_rates

Amikacin                     0.0
Amoxycillin clavulanate      0.0
Ampicillin                   0.0
Azithromycin                 0.0
Cefepime                     0.0
Cefoxitin                    0.0
Ceftazidime                  0.0
Ceftriaxone                  0.0
Clarithromycin               0.0
Clindamycin                  0.0
Erythromycin                 0.0
Imipenem                     0.0
Levofloxacin                 0.0
Linezolid                    0.0
Meropenem                    0.0
Metronidazole                0.0
Minocycline                  0.0
Penicillin                   0.0
Piperacillin tazobactam      0.0
Tigecycline                  0.0
Vancomycin                   0.0
Ampicillin sulbactam         0.0
Aztreonam                    0.0
Aztreonam avibactam          0.0
Cefixime                     0.0
Ceftaroline                  0.0
Ceftaroline avibactam        0.0
Ceftazidime avibactam        0.0
Ciprofloxacin                0.0
Colistin                     0.0
Daptomycin

In [103]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Load the dataset
def load_data(file_path):
    df = pd.read_csv(file_path, low_memory=False)
    df.columns = df.columns.str.replace(' ', '.')
    return df

# Filter, subgroup data, convert to long format, clean the data, and calculate resistance percentage
def filter_convert_and_calculate(df, antibiotics):
    result = {}
    for antibiotic in antibiotics:
        if antibiotic in df.columns:
            # Filter rows where the antibiotic is marked as 'Resistant'
            resistant_df = df[df[antibiotic] == 'Resistant']
            
            # Exclude the antibiotic of interest and convert to long format
            long_df = resistant_df.drop(columns=[antibiotic])
            long_df = pd.melt(long_df, id_vars=['Species', 'Family', 'Country', 'Gender', 'Age.Group', 'Speciality', 'Source', 'In./.Out.Patient', 'Year'], 
                              var_name='Antibiotics', value_name='Resistance')
            
            # Remove rows with NaN values in 'Resistance'
            long_df = long_df.dropna(subset=['Resistance'])
            
            # Calculate the percentage of 'Resistant' for each Antibiotic
            resistance_percent = long_df.groupby('Antibiotics')['Resistance'].apply(lambda x: (x == 'Resistant').mean() * 100).reset_index()
            resistance_percent.rename(columns={'Resistance': 'Resistance_Percentage'}, inplace=True)
            
            result[antibiotic] = resistance_percent
    return result

# Combine all resistance percentages into a single DataFrame for heatmap
def combine_resistance_percentages(resistance_data):
    combined_df = pd.DataFrame()
    for antibiotic, data in resistance_data.items():
        if not data.empty:
            data.set_index('Antibiotics', inplace=True)
            combined_df = pd.concat([combined_df, data.rename(columns={'Resistance_Percentage': antibiotic})], axis=1)
    
    # Sort columns and index alphabetically
    combined_df = combined_df.sort_index().sort_index(axis=1)
    
    return combined_df

# Generate a heatmap and save it
def plot_heatmap(df, save_path):
    plt.figure(figsize=(12, 10))
    sns.heatmap(df, cmap='RdYlGn_r', cbar=True, linewidths=.5, annot=False, 
                mask=df.isnull(),  # Mask missing values
                linecolor='grey',  # Color for the lines between cells
                edgecolor='grey')  # Color for edges of cells
    plt.title('Heatmap of Resistance Percentages Across Antibiotics')
    plt.xlabel('Resistant First Line Antibiotic')
    plt.ylabel('Possible Second Line Antibiotic')
    
    # Save the figure
    plt.savefig(save_path, bbox_inches='tight')
    plt.close()

# Define the list of antibiotics
antibiotics = [
    'Amikacin', 'Amoxycillin.clavulanate', 'Ampicillin', 'Azithromycin', 'Cefepime', 
    'Cefoxitin', 'Ceftazidime', 'Ceftriaxone', 'Clarithromycin', 'Clindamycin', 
    'Erythromycin', 'Imipenem', 'Levofloxacin', 'Linezolid', 'Meropenem', 
    'Metronidazole', 'Minocycline', 'Penicillin', 'Piperacillin.tazobactam', 
    'Tigecycline', 'Vancomycin', 'Ampicillin.sulbactam', 'Aztreonam', 
    'Aztreonam.avibactam', 'Cefixime', 'Ceftaroline', 'Ceftaroline.avibactam', 
    'Ceftazidime.avibactam', 'Ciprofloxacin', 'Colistin', 'Daptomycin', 
    'Doripenem', 'Ertapenem', 'Gatifloxacin', 'Gentamicin', 'Moxifloxacin', 
    'Oxacillin', 'Quinupristin.dalfopristin', 'Sulbactam', 'Teicoplanin', 
    'Tetracycline', 'Trimethoprim.sulfa', 'Ceftolozane.tazobactam', 
    'Cefoperazone.sulbactam', 'Meropenem.vaborbactam', 'Cefpodoxime', 
    'Ceftibuten', 'Ceftibuten.avibactam', 'Tebipenem'
]

# Load data
file_path = 'wide_atlas.csv'
df = load_data(file_path)

# Filter, subgroup, convert to long format, clean the data, and calculate resistance percentage
resistance_percentage_data = filter_convert_and_calculate(df, antibiotics)

# Combine all resistance percentages into a single DataFrame
combined_resistance_df = combine_resistance_percentages(resistance_percentage_data)

# Path to save the heatmap
save_path = r'heatmap_resistance_percentages.png'

# Plot and save the combined heatmap
plot_heatmap(combined_resistance_df, save_path)


In [104]:
combined_resistance_df.to_csv('esclating_antibiogram.csv')

In [91]:
# Filter, subgroup, convert to long format, clean the data, and calculate resistance percentage
resistance_percentage_data = filter_convert_and_calculate(df, antibiotics)


In [92]:
# Example: Display the resistance percentage DataFrame for a specific antibiotic (e.g., 'Amikacin')
resistance_percentage_data['Amikacin']

,Antibiotics,Resistance_Percentage
0,Amikacin,100.000000
1,Amoxycillin.clavulanate,71.136364
2,Ampicillin,97.500000
3,Ampicillin.sulbactam,75.139912
4,Aztreonam,72.289157
5,Cefepime,87.478043
6,Cefixime,89.285714
7,Cefpodoxime,91.565350
8,Ceftaroline,93.636364
9,Ceftazidime,89.636136


In [6]:
# Define the list of antibiotics
antibiotics = [
    'Amikacin', 'Amoxycillin clavulanate', 'Ampicillin', 'Azithromycin', 'Cefepime', 
    'Cefoxitin', 'Ceftazidime', 'Ceftriaxone', 'Clarithromycin', 'Clindamycin', 
    'Erythromycin', 'Imipenem', 'Levofloxacin', 'Linezolid', 'Meropenem', 
    'Metronidazole', 'Minocycline', 'Penicillin', 'Piperacillin.tazobactam', 
    'Tigecycline', 'Vancomycin', 'Ampicillin sulbactam', 'Aztreonam', 
    'Aztreonam avibactam', 'Cefixime', 'Ceftaroline', 'Ceftaroline.avibactam', 
    'Ceftazidime avibactam', 'Ciprofloxacin', 'Colistin', 'Daptomycin', 
    'Doripenem', 'Ertapenem', 'Gatifloxacin', 'Gentamicin', 'Moxifloxacin', 
    'Oxacillin', 'Quinupristin.dalfopristin', 'Sulbactam', 'Teicoplanin', 
    'Tetracycline', 'Trimethoprim.sulfa', 'Ceftolozane.tazobactam', 
    'Cefoperazone sulbactam', 'Meropenem vaborbactam', 'Cefpodoxime', 
    'Ceftibuten', 'Ceftibuten avibactam', 'Tebipenem'
]

In [10]:
import requests
import pandas as pd

# Define the base URLs for the FDA API
base_url_side_effects = "https://api.fda.gov/drug/event.json"
indications_url = "https://api.fda.gov/drug/label.json"

# Load the AWARE dataset
aware_df = pd.read_csv('WHO_AWARE.csv')

# Define the list of antibiotics
antibiotics = [
    'Amikacin', 'Amoxycillin clavulanate', 'Ampicillin', 'Azithromycin', 'Cefepime', 
    'Cefoxitin', 'Ceftazidime', 'Ceftriaxone', 'Clarithromycin', 'Clindamycin', 
    'Erythromycin', 'Imipenem', 'Levofloxacin', 'Linezolid', 'Meropenem', 
    'Metronidazole', 'Minocycline', 'Penicillin', 'Piperacillin.tazobactam', 
    'Tigecycline', 'Vancomycin', 'Ampicillin sulbactam', 'Aztreonam', 
    'Aztreonam avibactam', 'Cefixime', 'Ceftaroline', 'Ceftaroline.avibactam', 
    'Ceftazidime avibactam', 'Ciprofloxacin', 'Colistin', 'Daptomycin', 
    'Doripenem', 'Ertapenem', 'Gatifloxacin', 'Gentamicin', 'Moxifloxacin', 
    'Oxacillin', 'Quinupristin.dalfopristin', 'Sulbactam', 'Teicoplanin', 
    'Tetracycline', 'Trimethoprim.sulfa', 'Ceftolozane.tazobactam', 
    'Cefoperazone sulbactam', 'Meropenem vaborbactam', 'Cefpodoxime', 
    'Ceftibuten', 'Ceftibuten avibactam', 'Tebipenem'
]


# Initialize an empty list to store questions and answers
combined_qa_dataset = []

# Function to fetch side effects for a specific antibiotic
def fetch_side_effects(antibiotic):
    params = {
        'search': f'patient.drug.medicinalproduct:{antibiotic}',
        'count': 'patient.reaction.reactionmeddrapt.exact',
        'limit': 5  # Limit to top 5 side effects
    }
    
    response = requests.get(base_url_side_effects, params=params)
    
    if response.status_code == 200:
        results = response.json().get('results', [])
        return [result['term'] for result in results]
    else:
        return []

# Function to fetch indications for a specific antibiotic
def fetch_indications(antibiotic):
    params = {
        'search': f'openfda.generic_name:{antibiotic}',
        'limit': 1  # Get the most relevant result
    }
    
    response = requests.get(indications_url, params=params)
    
    if response.status_code == 200:
        results = response.json().get('results', [])
        if results:
            return results[0].get('indications_and_usage', [])
    return []

# Loop over each antibiotic and generate questions and answers for side effects, AWARE status, and indications
for antibiotic in antibiotics:
    # Fetch side effects
    side_effects = fetch_side_effects(antibiotic)
    
    # Fetch AWARE status
    aware_status = aware_df[aware_df['Antibiotic'].str.contains(antibiotic, case=False, na=False)]
    
    # Fetch indications
    indications = fetch_indications(antibiotic)
    
    # Add side effects question and answer if available
    if side_effects:
        question_side_effects = f"What are the common side effects of {antibiotic}?"
        answer_side_effects = ', '.join(side_effects)
        combined_qa_dataset.append({'Question': question_side_effects, 'Answer': answer_side_effects})
    
    # Add AWARE status question and answer if available
    if not aware_status.empty:
        status = aware_status.iloc[0]['Category']
        question_aware = f"What is the AWARE status of {antibiotic}?"
        answer_aware = f"{status}"
        combined_qa_dataset.append({'Question': question_aware, 'Answer': answer_aware})
    
    # Add indications question and answer if available
    if indications:
        question_indications = f"What are the indications for {antibiotic}?"
        answer_indications = indications[0] if isinstance(indications, list) else indications
        combined_qa_dataset.append({'Question': question_indications, 'Answer': answer_indications})

# Convert the combined dataset to a DataFrame
combined_df = pd.DataFrame(combined_qa_dataset)




In [11]:
combined_df

,Question,Answer
0,What are the common side effects of Amikacin?,"DRUG INEFFECTIVE, OFF LABEL USE, PYREXIA, DRUG..."
1,What is the AWARE status of Amikacin?,Access
2,What are the indications for Amikacin?,INDICATIONS AND USAGE Amikacin Sulfate Injecti...
3,What are the common side effects of Amoxycilli...,"DIARRHOEA, DYSPNOEA, NAUSEA, PYREXIA, DRUG HYP..."
4,What are the indications for Amoxycillin clavu...,INDICATIONS AND USAGE Clarithromycin tablets a...
...,...,...
118,What is the AWARE status of Ceftibuten?,Watch
119,What are the common side effects of Ceftibuten...,"OFF LABEL USE, DRUG INEFFECTIVE, DEATH, PATHOG..."
120,What are the indications for Ceftibuten avibac...,1. INDICATIONS AND USAGE EXBLIFEP is a combina...
121,What are the common side effects of Tebipenem?,"BLOOD LACTIC ACID INCREASED, CLOSTRIDIUM DIFFI..."


In [12]:
# Optionally, save the dataset to a CSV file
combined_df.to_csv('antibiotics_qa.csv', index=False)

In [16]:
import os
import re
import pandas as pd
import requests
import streamlit as st
from dotenv import load_dotenv

# Load environment variables from the .env file
load_dotenv()

# Define the list of antibiotics
antibiotics = [
    'Amikacin', 'Amoxycillin clavulanate', 'Ampicillin', 'Azithromycin', 'Cefepime', 
    'Cefoxitin', 'Ceftazidime', 'Ceftriaxone', 'Clarithromycin', 'Clindamycin', 
    'Erythromycin', 'Imipenem', 'Levofloxacin', 'Linezolid', 'Meropenem', 
    'Metronidazole', 'Minocycline', 'Penicillin', 'Piperacillin tazobactam', 
    'Tigecycline', 'Vancomycin', 'Ampicillin sulbactam', 'Aztreonam', 
    'Aztreonam avibactam', 'Cefixime', 'Ceftaroline', 'Ceftaroline avibactam', 
    'Ceftazidime avibactam', 'Ciprofloxacin', 'Colistin', 'Daptomycin', 
    'Doripenem', 'Ertapenem', 'Gatifloxacin', 'Gentamicin', 'Moxifloxacin', 
    'Oxacillin', 'Quinupristin dalfopristin', 'Sulbactam', 'Teicoplanin', 
    'Tetracycline', 'Trimethoprim sulfa', 'Ceftolozane tazobactam', 
    'Cefoperazone sulbactam', 'Meropenem vaborbactam', 'Cefpodoxime', 
    'Ceftibuten', 'Ceftibuten avibactam', 'Tebipenem'
]

# Initialize a list to store all the antibiotic data
all_fda_data = []

# Loop over each antibiotic to fetch the FDA information
for antibiotic_info in antibiotics:
    # Fetch Drug Labeling Information
    base_url_label = "https://api.fda.gov/drug/label.json"
    params_label = {
        "search": f"openfda.brand_name:{antibiotic_info.lower()}",
        "limit": 1
    }
    response_label = requests.get(base_url_label, params=params_label)
    if response_label.status_code == 200:
        label_data = response_label.json()
        if label_data['results']:
            label_output = {
                "Antibiotic": antibiotic_info,
                "Brand Name": label_data['results'][0].get('openfda', {}).get('brand_name', ['N/A'])[0],
                "Generic Name": label_data['results'][0].get('openfda', {}).get('generic_name', ['N/A'])[0],
                "Indications and Usage": label_data['results'][0].get('indications_and_usage', ['N/A'])[0]
            }
        else:
            label_output = {
                "Antibiotic": antibiotic_info,
                "Brand Name": "N/A",
                "Generic Name": "N/A",
                "Indications and Usage": "N/A"
            }
    else:
        label_output = {
            "Antibiotic": antibiotic_info,
            "Brand Name": "Failed to retrieve",
            "Generic Name": "Failed to retrieve",
            "Indications and Usage": "Failed to retrieve"
        }

    # Fetch Adverse Events (Side Effects) Information
    base_url_event = "https://api.fda.gov/drug/event.json"
    params_event = {
        "search": f"patient.drug.medicinalproduct:{antibiotic_info.lower()}",
        "count": "patient.reaction.reactionmeddrapt.exact",
        "limit": 10
    }
    response_event = requests.get(base_url_event, params=params_event)
    if response_event.status_code == 200:
        event_data = response_event.json()
        if 'results' in event_data:
            side_effects = [event['term'] for event in event_data['results']]
        else:
            side_effects = ["No adverse events information available"]
    else:
        side_effects = ["Failed to retrieve adverse events information"]

    # Add the side effects to the label_output dictionary
    label_output["Side Effects"] = side_effects

    # Append the output to the list
    all_fda_data.append(label_output)

# Convert the list of dictionaries into a DataFrame
df_fda = pd.DataFrame(all_fda_data)



In [17]:
# Save the DataFrame to a CSV file
df_fda.to_csv('FDA.csv', index=False)
st.write("FDA information for all antibiotics has been saved to 'FDA.csv'.")


In [18]:
df_fda

,Antibiotic,Brand Name,Generic Name,Indications and Usage,Side Effects
0,Amikacin,Amikacin Sulfate,AMIKACIN SULFATE,INDICATIONS AND USAGE Amikacin Sulfate Injecti...,"[DRUG INEFFECTIVE, OFF LABEL USE, PYREXIA, DRU..."
1,Amoxycillin clavulanate,N/A,N/A,INDICATIONS AND USAGE Clarithromycin tablets a...,"[DIARRHOEA, DYSPNOEA, NAUSEA, PYREXIA, DRUG HY..."
2,Ampicillin,Ampicillin and Sulbactam,AMPICILLIN SODIUM AND SULBACTAM SODIUM,INDICATIONS AND USAGE Ampicillin and sulbactam...,"[DRUG INEFFECTIVE, DRUG HYPERSENSITIVITY, FOET..."
3,Azithromycin,Azithromycin,AZITHROMYCIN,1 INDICATIONS AND USAGE Azithromycin is a macr...,"[OFF LABEL USE, CHRONIC KIDNEY DISEASE, DRUG I..."
4,Cefepime,Cefepime,CEFEPIME,1 INDICATIONS AND USAGE Cefepime for Injection...,"[ACUTE KIDNEY INJURY, DRUG INEFFECTIVE, PYREXI..."
5,Cefoxitin,Cefoxitin,CEFOXITIN SODIUM,INDICATIONS AND USAGE: Treatment: CEFOXITIN FO...,[DRUG REACTION WITH EOSINOPHILIA AND SYSTEMIC ...
6,Ceftazidime,Ceftazidime,CEFTAZIDIME,INDICATIONS AND USAGE Ceftazidime for injectio...,"[DRUG INEFFECTIVE, OFF LABEL USE, PYREXIA, SEP..."
7,Ceftriaxone,Ceftriaxone Sodium,CEFTRIAXONE SODIUM,INDICATIONS AND USAGE Before instituting treat...,"[OFF LABEL USE, DRUG INEFFECTIVE, PYREXIA, ACU..."
8,Clarithromycin,Clarithromycin,CLARITHROMYCIN,1 INDICATIONS AND USAGE Clarithromycin is a ma...,"[DRUG INTERACTION, DRUG INEFFECTIVE, NAUSEA, D..."
9,Clindamycin,Clindamycin Phosphate,CLINDAMYCIN PHOSPHATE,INDICATIONS AND USAGE Clindamycin phosphate to...,"[DRUG HYPERSENSITIVITY, DRUG INEFFECTIVE, DIAR..."
